In [1]:
import qiskit.extensions.simulator  
from qiskit import (QuantumRegister, ClassicalRegister,QuantumCircuit,
                   get_backend,compile)

import os 
import sys 
import unittest
import logging
logger = logging.Logger('error')
sys.path.append('../')

In [2]:
from qiskit import store_credentials
import json
with open(os.environ['QCONFIG'],'r') as f:
    local_config = json.loads(f.read())

qe_token = local_config['APItoken']
qe_url = local_config['config']['url']
backends = ['ibmqx_hpc_qasm_simulator','ibmq_qasm_simulator'] 

In [3]:
from qiskit.backends.ibmq import IBMQProvider
from python.common import Path, QiskitTestCase, requires_qe_access, slow_test


class TestBackendQobj(QiskitTestCase):
    """Qiskit backend qobj test."""
    

    def test_simple_circuit(self):
        """Test one circuit, one register, in-order readout.
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr[0], cr[0])
        circ.measure(qr[1], cr[1])
        circ.measure(qr[2], cr[2])
        circ.measure(qr[3], cr[3])
        local = get_backend('local_qasm_simulator')
        ibmq_provider = IBMQProvider(qe_token, qe_url)
        remotes = ibmq_provider.available_backends()
        for remote in remotes:
            #self.log.info(remote.status())
            if remote.configuration()['name'] == backend:
                self.assertTrue(remote.status()['operational'])
                self.assertTrue(remote.configuration()['allow_q_object'])
                qobj = compile(circ, remote)
                result_remote = remote.run(qobj).result()
                result_local = local.run(qobj).result()
                self.assertDictAlmostEqual(result_remote.get_counts(circ),
                                      result_local.get_counts(circ))


    def test_readout_order(self):
        """Test one circuit, one register, out-of-order readout.
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr[0], cr[2])
        circ.measure(qr[1], cr[0])
        circ.measure(qr[2], cr[1])
        circ.measure(qr[3], cr[3])
        local = get_backend('local_qasm_simulator')
        ibmq_provider = IBMQProvider(qe_token, qe_url)
        remotes = ibmq_provider.available_backends()
        for remote in remotes:
            #self.log.info(remote.status())
            if remote.configuration()['name'] == backend:
                self.assertTrue(remote.status()['operational'])
                self.assertTrue(remote.configuration()['allow_q_object'])
                qobj = compile(circ, remote)
                result_remote = remote.run(qobj).result()
                result_local = local.run(qobj).result()
                self.assertDictAlmostEqual(result_remote.get_counts(circ),
                                      result_local.get_counts(circ))

                
    def test_multi_register(self):
        """Test one circuit, two registers, out-of-order readout.
        """
        qr1 = QuantumRegister(4)
        qr2 = QuantumRegister(2)
        cr1 = ClassicalRegister(3)
        cr2 = ClassicalRegister(1)
        circ = QuantumCircuit(qr1, qr2, cr1, cr2)
        circ.h(qr1[0])
        circ.cx(qr1[0], qr2[1])
        circ.h(qr2[0])
        circ.cx(qr2[0], qr1[3])
        circ.x(qr1[1])
        circ.measure(qr1[0], cr2[0])
        circ.measure(qr1[1], cr1[0])
        circ.measure(qr1[2], cr2[0])
        circ.measure(qr1[3], cr1[2])
        circ.measure(qr2[0], cr1[2])
        circ.measure(qr2[1], cr1[1])
        local = get_backend('local_qasm_simulator')
        ibmq_provider = IBMQProvider(qe_token, qe_url)
        remotes = ibmq_provider.available_backends()
        for remote in remotes:
            #self.log.info(remote.status())
            if remote.configuration()['name'] == backend:
                self.assertTrue(remote.status()['operational'])
                self.assertTrue(remote.configuration()['allow_q_object'])
                qobj = compile(circ, remote)
                result_remote = remote.run(qobj).result()
                result_local = local.run(qobj).result()
                self.assertDictAlmostEqual(result_remote.get_counts(circ),
                                      result_local.get_counts(circ),delta=50)

    def test_multi_circuit(self):
        """Test two circuits, two registers, out-of-order readout.
        """
        qr1 = QuantumRegister(4)
        qr2 = QuantumRegister(2)
        cr1 = ClassicalRegister(3)
        cr2 = ClassicalRegister(1)
        circ1 = QuantumCircuit(qr1, qr2, cr1, cr2)
        circ1.h(qr1[0])
        circ1.cx(qr1[0], qr2[1])
        circ1.h(qr2[0])
        circ1.cx(qr2[0], qr1[3])
        circ1.x(qr1[1])
        circ1.measure(qr1[0], cr2[0])
        circ1.measure(qr1[1], cr1[0])
        circ1.measure(qr1[2], cr2[0])
        circ1.measure(qr1[3], cr1[2])
        circ1.measure(qr2[0], cr1[2])
        circ1.measure(qr2[1], cr1[1])
        circ2 = QuantumCircuit(qr1, qr2, cr1)
        circ2.h(qr1[0])
        circ2.cx(qr1[0], qr1[3])
        circ2.h(qr2[1])
        circ2.cx(qr2[1], qr1[3])
        circ2.measure(qr1[0], cr1[0])
        circ2.measure(qr1[1], cr1[1])
        circ2.measure(qr1[2], cr1[2])
        circ2.measure(qr2[1], cr1[2])
        local = get_backend('local_qasm_simulator')
        ibmq_provider = IBMQProvider(qe_token, qe_url)
        remotes = ibmq_provider.available_backends()
        for remote in remotes:
            #self.log.info(remote.status())
            if remote.configuration()['name'] == backend:
                    self.assertTrue(remote.status()['operational'])
                    self.assertTrue(remote.configuration()['allow_q_object'])
                 
                    qobj = compile([circ1, circ2], remote)
                    result_remote = remote.run(qobj).result()
                    result_local = local.run(qobj).result()
                    self.assertDictAlmostEqual(result_remote.get_counts(circ1),
                                          result_local.get_counts(circ1),delta=50)
                    self.assertDictAlmostEqual(result_remote.get_counts(circ2),
                                          result_local.get_counts(circ2),delta=50)
    
    def test_conditional_operation(self):
        """Test conditional operation. 
        """
        qr = QuantumRegister(4)
        cr = ClassicalRegister(4)
        circ = QuantumCircuit(qr, cr)
        circ.x(qr[0])
        circ.x(qr[2])
        circ.measure(qr[0], cr[0])
        circ.x(qr[0]).c_if(cr,1)
        local = get_backend('local_qasm_simulator')
        ibmq_provider = IBMQProvider(qe_token, qe_url)
        remotes = ibmq_provider.available_backends()
        for remote in remotes:
            #self.log.info(remote.status())
            if remote.configuration()['name'] == backend:
                self.assertTrue(remote.status()['operational'])
                self.assertTrue(remote.configuration()['allow_q_object'])
                qobj = compile(circ, remote)
                result_remote = remote.run(qobj).result()
                result_local = local.run(qobj).result()
                self.assertDictAlmostEqual(result_remote.get_counts(circ),
                                      result_local.get_counts(circ))


In [4]:
#normal qasm simulator 
backend = backends[1]
testbackend = TestBackendQobj()
test_methods = [testbackend.test_multi_circuit,testbackend.test_multi_register,
           testbackend.test_readout_order,testbackend.test_simple_circuit,testbackend.test_conditional_operation]

failed = 0 
for test in test_methods:
    try:
        test()
        print('.',end='')
    except Exception as err:
        failed += 1
        print('x',end='')
        logger.warning(err)
        import pdb;pdb.set_trace()
        logger.error('test failed:{}'.format(str(test)))
print('\n {0} of {1} tests failed'.format(failed,len(test_methods)))

.....
 0 of 5 tests failed


In [5]:
#hpc simulator
backend = backends[0]

failed = 0 
for test in test_methods:
    try:
        test()
        print('.',end='')
    except Exception as err:
        failed += 1
        print('x',end='')
        logger.warning(err)
        logger.error('test failed:{}'.format(str(test)))
print('\n {0} of {1} tests failed'.format(failed,len(test_methods)))

Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Couldn't get backend status: Expecting value: line 1 column 1 (char 0)
test failed:<bound method TestBackendQobj.test_multi_circuit of <__main__.TestBackendQobj testMethod=runTest>>


x

Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Couldn't get backend status: Expecting value: line 1 column 1 (char 0)
test failed:<bound method TestBackendQobj.test_multi_register of <__main__.TestBackendQobj testMethod=runTest>>


x

Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Couldn't get backend status: Expecting value: line 1 column 1 (char 0)
test failed:<bound method TestBackendQobj.test_readout_order of <__main__.TestBackendQobj testMethod=runTest>>


x

Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Couldn't get backend status: Expecting value: line 1 column 1 (char 0)
test failed:<bound method TestBackendQobj.test_simple_circuit of <__main__.TestBackendQobj testMethod=runTest>>


x

Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Got a 204 code response to https://q-console-api-staging.mybluemix.net/api/Backends/ibmqx_hpc_qasm_simulator/queue/status: 
Couldn't get backend status: Expecting value: line 1 column 1 (char 0)
test failed:<bound method TestBackendQobj.test_conditional_operation of <__main__.TestBackendQobj testMethod=runTest>>


x
 5 of 5 tests failed
